<div style='background: #FFE7DF; border-radius:10px; taxt-align: center; width: 444px; margin: 55px auto; padding:44px; border: 1px solid #754F4F'>
    <h5 style=' text-align:center; margin: auto'>
        Проведём анализ опроса
    </h5>
</div>

In [1]:
import pandas as pd
import numpy as np
import altair as alt

In [2]:
df_origin = pd.read_excel('f/ИССЛЕДОВАНИЕ ПОВЕДЕНИЯ (Ответы).xlsx')

# размер

In [3]:
df_origin.shape

(104, 12)

In [48]:
df_origin['Пол'].value_counts()

Женский    78
Мужской    26
Name: Пол, dtype: int64

###### преобразуем таблицу 

In [4]:
# Считаем новый столбец с возрастной группой 
def group(col):
    if col < 25:
        val = 1
    elif (col > 25) & (col < 35):
        val = 2
    elif (col > 35) & (col < 50):
        val = 3
    else :
        val = 4
    return str(val)

In [5]:
# определяем время дня
def time(col):
    if col < 12:
        time = 'утро'
    elif 12 < col < 17:
        time = 'день'
    elif 17 < col < 22:
        time = 'вечер'
    else:
        time = 'ночь'
    return time

In [6]:
df_origin['Возрастная категория'] = df_origin['Укажите ваш возраст'].apply (group)

In [7]:
# Меняем значение в столбце на более короткое
df_origin.loc[df_origin['Выберете сферу вашей деятельности'] == 'Искусство и культура (кино, музыкальный бизнес, музеи и галереи, и т. д.)', \
       'Выберете сферу вашей деятельности'] = 'Сфера искусства'

In [8]:
# дата
# df_origin.drop(['Отметка времени'], axis=1, inplace=True)

In [9]:
# переименовываем столбцы 
df_origin.rename(columns = {'Отметка времени': 'Дата', 'Выберете пол': 'Пол', 'Женат / замужем': 'Семья',
                    'Укажите ваш возраст': 'Возраст','Есть ли у вас дети?': 'Дети', 
                     'Выберете сферу вашей деятельности': 'Деятельность', 
                     'Оцените, как часто вы пользуетесь  социальными сетями?\nОтветы даются по пятибалльной шкале:\n1 – очень редко\n2 – иногда\n3 – часто\n4 – очень часто\n5 – практически не выхожу': 'Частота использования',
                     'Перейдёте по ссылке по рекомендации друзей/знакомых?': 'Переход по ссылке', 
                     'Сделаете репост по просьбе друзей/знакомых?': 'Репост по просьбе', 'Какие чувства вызовет такая реклама услуг?': 'Что вызывает реклама услуг', 'Какие чувства вызовет просьба сделать репост?': 'Чувства от просьбы'
                    }, inplace = True )


### разбивают дату – день, час, время дня 

In [10]:
# df_origin['День месяца'] = df_origin['Дата'].dt.day 
df_origin.insert(1, "День месяца", df_origin['Дата'].dt.day)
df_origin.insert(2, "Час прохождения опроса", df_origin['Дата'].dt.hour)
df_origin.insert(2, "Время дня", df_origin['Час прохождения опроса'].apply (time))

In [11]:
df_origin['Время'] = df_origin['Час прохождения опроса'].apply (time)

In [12]:
df_origin.sample(3)

,Дата,День месяца,Время дня,Час прохождения опроса,Пол,Возраст,Семья,Дети,Высшее образование,Деятельность,Частота использования,Переход по ссылке,Репост по просьбе,Что вызывает реклама услуг,Чувства от просьбы,Возрастная категория,Время
10,2023-03-10 18:23:10.934,10,вечер,18,Мужской,40,Да,Да,Да,Сфера обслуживания,4,Да,Нет,Равнодушие,Меня используют (манипуляция),3,вечер
93,2023-03-17 08:03:26.157,17,утро,8,Женский,50,Да,Да,Да,Бухгалтерия / Финансы / Аудит / Страхование,3,Да,Да,Интерес,Желание помочь,4,утро
94,2023-03-17 14:23:05.968,17,день,14,Мужской,36,Нет,Нет,Да,"IT, телеком, диджитал-сервисы",1,Нет,Нет,Равнодушие,Равнодушие,3,день


## количество вопросов по дням

In [27]:
df_origin.groupby('День месяца').agg({'Пол': 'count'})

,Пол
День месяца,
10,12
11,18
12,13
13,23
14,15
...,...
19,1
20,1
21,1


#  разбиваю на мужчин и женщин, обрезаю лишнее, объединяю в одну таблицу 

In [14]:
woman = df_origin[df_origin['Пол'] == 'Женский'].reset_index()
man = df_origin[df_origin['Пол'] == 'Мужской'].reset_index()

In [15]:
if len(woman) > len(man):
    i = len(woman) - len(man)
elif len(woman) < len(man):
    i = len(man) - len(woman)
else:
    i = 0
for j in range(i+1):
    if len(woman) > len(man):
        woman = woman.drop (index = 0 )
        woman.reset_index(drop=True, inplace=True)
    elif len(woman) < len(man):
        man = man.drop (index = 0 )
        man.reset_index(drop=True, inplace=True)
    else:
        print(len(man), len(woman))

26 26


In [16]:
frames = [woman, man]
df = pd.concat(frames)
df.reset_index(drop=True, inplace=True)
df.drop(['index'], axis=1, inplace=True)
df.sample(3)

,Дата,День месяца,Время дня,Час прохождения опроса,Пол,Возраст,Семья,Дети,Высшее образование,Деятельность,Частота использования,Переход по ссылке,Репост по просьбе,Что вызывает реклама услуг,Чувства от просьбы,Возрастная категория,Время
45,2023-03-16 16:16:44.019,16,день,16,Мужской,21,Да,Да,Да,Министеоство Внутренних Дел,4,Да,Да,Интерес,Желание помочь,1,день
35,2023-03-12 11:59:16.042,12,утро,11,Мужской,48,Да,Да,Да,Бухгалтерия / Финансы / Аудит / Страхование,2,Да,Да,Меня используют (манипуляция),Стыд / неловкость / что подумают обо мне получатели,3,утро
16,2023-03-16 17:05:22.755,16,ночь,17,Женский,24,Да,Да,Да,Не работаю,4,Да,Да,Всегда по разному.,Заинтересованность,1,ночь


In [25]:
df_origin['Возрастная категория'] = df_origin['Возраст'].apply (group)

<b style='font-family: bold'>Возрастная категория</b>
<ol>
  <li> - До 25 лет</li>
  <li> - От 25 до 35 лет</li>
  <li> - От 35 до 50 лет</li>
  <li> - После 50 лет</li>
</ol>

In [22]:
# Меняем значение в столбце на более короткое
df_origin.loc[df_origin['Деятельность'] == 'Искусство и культура (кино, музыкальный бизнес, музеи и галереи, и т. д.)', \
       'Выберете сферу вашей деятельности'] = 'Сфера искусства'


In [23]:
# Удаляем дату
# df_origin.drop(['Отметка времени'], axis=1, inplace=True)
# df_origin['День месяца'] = df_origin['Дата'].dt.day 

# Предлагаю познакомиться c данными
### посмотрим, как выглядит наша аудитория 

In [28]:
df_origin.groupby(['День месяца']).agg(
    Количество=pd.NamedAgg(column="Частота использования", aggfunc='size'),
    Сумма=pd.NamedAgg(column="Частота использования", aggfunc='sum')
).round(2)


,Количество,Сумма
День месяца,,
10,12,36
11,18,61
12,13,44
13,23,77
14,15,53
...,...,...
19,1,2
20,1,2
21,1,5


In [29]:
man.describe().style.format({'Возраст': '{0:,.0f}', 'Частота использования': '{0:,.0f}'})

,День месяца,Час прохождения опроса,Возраст,Частота использования
count,26.000000,26.000000,26,26
mean,13.884615,13.538462,37,3
std,3.723728,4.120493,10,1
min,10.000000,1.000000,19,1
25%,10.250000,11.000000,30,2
50%,13.000000,14.000000,36,3
75%,16.000000,16.000000,42,4
max,22.000000,21.000000,60,5


## смотрим среднее значение частоты использования по полу, семейному положению, сферы деятельности 

In [30]:
table = pd.pivot_table(df.round({'Частота использования': 0}), columns=["Пол","Семья"],index="Деятельность",
                       values="Частота использования", aggfunc={"Частота использования": np.mean},fill_value=0)
table.round(1)

Пол                                         Женский      Мужской     
Семья                                            Да  Нет      Да  Нет
Деятельность                                                         
IT, телеком, диджитал-сервисы                   4.0  3.7     2.0  2.8
Бухгалтерия / Финансы / Аудит / Страхование     3.2  3.0     2.0  0.0
Маркетинговые, рекламные агентства              5.0  0.0     0.0  0.0
Медицина и фармакология                         0.0  0.0     3.0  4.0
Министеоство Внутренних Дел                     0.0  0.0     4.0  0.0
Не работаю                                      4.0  0.0     1.0  2.0
Предприниматель                                 0.0  0.0     5.0  0.0
Производство, промышленность                    2.5  0.0     2.8  3.0
Ритейл, ecommerce                               3.0  0.0     0.0  3.0
СМИ, медиа и издательства                       4.0  0.0     0.0  0.0
Сфера искусства                                 0.0  0.0     4.0  0.0
Сфера образования                               5.0  0.0     2.0  0.0
Сфера обслуживания                              0.0  3.5     2.5  0.0
Транспорт и логистика                           5.0  0.0     0.0  0.0
экономика                                       5.0  0.0     0.0  0.0


<div style='background: #C5E8EF; taxt-align: center; width: 444px; margin: auto; padding:44px; border: 1px solid #636363'>
    <h5 style='color: #476285;  text-align:center; margin: auto'>
        Выводы 
    </h5>
</div>

###  Количество определенных возрастных категорий в каждой отрасли деятельности 

In [31]:
pivot_count = pd.pivot_table(man, 'Частота использования', 'Деятельность', 'Возрастная категория', 
                             aggfunc=['count'])

In [32]:
pivot_count

count               
Возрастная категория                            1    2    3    4
Деятельность                                                    
IT, телеком, диджитал-сервисы                 NaN  2.0  3.0  1.0
Бухгалтерия / Финансы / Аудит / Страхование   NaN  NaN  1.0  NaN
Медицина и фармакология                       1.0  NaN  NaN  1.0
Министеоство Внутренних Дел                   1.0  NaN  NaN  NaN
Не работаю                                    1.0  NaN  NaN  1.0
Предприниматель                               NaN  NaN  1.0  NaN
Производство, промышленность                  NaN  3.0  3.0  NaN
Ритейл, ecommerce                             NaN  NaN  1.0  NaN
Сфера искусства                               NaN  NaN  NaN  1.0
Сфера образования                             NaN  NaN  1.0  NaN
Сфера обслуживания                            NaN  NaN  4.0  NaN


<div style='background: #C5E8EF; taxt-align: center; width: 444px; margin: auto; padding:44px; border: 1px solid #636363'>
    <h5 style='color: #476285;  text-align:center; margin: auto'>
        Выводы 
    </h5>
</div>

### посмотрим частоту использования соц сетей, рассматривая наличие семьи и вид деятельности  

In [34]:
df.groupby([df['Семья'], df['Деятельность']])['Частота использования'].agg(['min', 'median', 'max']).style.format({'median': '{0:,.0f}'})



<div style='background: #C5E8EF; taxt-align: center; width: 444px; margin: auto; padding:44px; border: 1px solid #636363'>
    <h5 style='color: #476285;  text-align:center; margin: auto'>
        яркая картина того, что у людей семьей меньше времени на то, чтобы тратить его на социальные сети  
    </h5>
</div>

In [ ]:
# all_income = list(df_origin['Что вызывает реклама услуг'])
# in_form = ['Желание помочь', 'Потребность поделиться', 'Стыд / неловкость / что подумают обо мне получатели', 
#               'Раздражение / агрессия', 'Меня используют (манипуляция)', 'Равнодушие']
# lst_= []

In [ ]:
# for id, val in enumerate(all_income):
#     if val not in in_form:
#         lst.append(val)
    

In [35]:
range_color1 = ['#040745', '#00F946']
               
chart1 = alt.Chart(df, title='Что вызывает рекламу услуг по возрасту').mark_bar().encode(
    x='Возраст:N',
    y='Что вызывает реклама услуг',
    color=alt.Color('Частота использования', scale=alt.Scale(range=range_color1)
))
chart1 = chart1.configure_title(fontSize=20, offset=5, orient='top', anchor='middle')

In [36]:
chart1

alt.Chart(...)

In [37]:
range_color2 = ['#F6EE83', '#8D0107']

chart2 = alt.Chart(df, title='Чувства от просьбы по возрасту').mark_bar().encode(
    x='Возраст:N',
    y='Чувства от просьбы',
    color=alt.Color('Частота использования', scale=alt.Scale(range=range_color2)
))
chart2 = chart2.configure_title(fontSize=20, offset=5, orient='top', anchor='middle')

In [38]:
chart2

alt.Chart(...)


<div style='background: #C5E8EF; taxt-align: center; width: 444px; margin: auto; padding:44px; border: 1px solid #636363'>
    <h5 style='color: #476285;  text-align:center; margin: auto'>
        Выводы 
    </h5>
</div>

In [39]:
range_color3 = ['#F6EE83', '#B5A59C', '#46587C', '#564423', '#55A020', '#905869', '#27B78D', '#50630F', 
                '#17ADBE', '#C26854', '#D6B8F5', '#8D0107', '#7A4D7A', '#E6676A', '#3D429C', '#E27AD6', 
                '#7B2BA8']

chart3 = alt.Chart(df, title='Чувства от просьбы в каждой должности').mark_bar().encode(
    x='Чувства от просьбы:O',
    y='sum(Возраст):Q',
    color=alt.Color('Частота использования:N', scale=alt.Scale(range=range_color3)),
    column='Деятельность:N'
) 
chart3 = chart3.configure_title(fontSize=20, offset=15, orient='top', anchor='start')

In [40]:
chart3

alt.Chart(...)

###  х

In [41]:
chart4 = alt.Chart(df, title='Есть ли взаимосвязь привязанности к социальным сетям от чувств от рекламных услуг')\
.mark_bar(size=50).encode(
        x='Частота использования:Q',
        y=alt.X('count()', axis=alt.Axis(title='Количество пользователей')),
        color=alt.Color('Что вызывает реклама услуг', scale=alt.Scale(range=range_color3))
    ).properties(
    width=800,
    height=300
)

chart4 = chart4.configure_title(fontSize=20, offset=45, orient='top', anchor='middle')

In [42]:
chart4

alt.Chart(...)


<div style='background: #225C68; taxt-align: center; width: 444px; margin: auto; padding:44px; border: 1px solid #636363'>
    <h5 style='color: #ffffff;  text-align:center; margin: auto'>
        тут что-нибудь написать про тех кто почти не вылезает из сот сетей, что они равнодушный к рекламе  
    </h5>
</div>

In [43]:

chart5 = alt.Chart(df, title='Влияют ли дети на наши чувства').mark_bar().encode(
        x='Чувства от просьбы:N',
        y=alt.X('count()', axis=alt.Axis(title='Количество')),
        color=alt.Color('Дети', scale=alt.Scale(range=range_color3))
    ).properties(
    width=800,
    height=300
)
chart5 = chart5.configure_title(fontSize=20, offset=5, orient='top', anchor='middle')

In [44]:
chart5

alt.Chart(...)

<div style='background: #FFE7DF; border-radius:10px; taxt-align: center; width: 444px; margin: 55px auto; padding:44px; border: 1px solid #754F4F'>
    <p style=' text-align:center; margin: auto'>
        Проведём анализ опроса
    </p>
</div>

In [45]:
# Меняем значение в столбце на более короткое
df_origin.loc[df_origin['Выберете сферу вашей деятельности'] == 'Искусство и культура (кино, музыкальный бизнес, музеи и галереи, и т. д.)', \
       'Выберете сферу вашей деятельности'] = 'Сфера искусства'


In [49]:
range_2 = ['#243384', '#E7C982']
chart = alt.Chart(df_origin).mark_bar().encode(
  y=alt.Y('Частота использования:Q'),
  x=alt.X('Возрастная категория:O'), 
  color=alt.Color('Пол:N', scale=alt.Scale(range=range_2)),
).facet(
  column=alt.Column('Деятельность:N'),
)
chart

alt.FacetChart(...)


<div style='background: #C5E8EF; taxt-align: center; width: 444px; margin: auto; padding:44px; border: 1px solid #636363'>
    <h5 style='color: #476285;  text-align:center; margin: auto'>
        Выводы 
    </h5>
</div>

In [53]:
base = alt.Chart(df).encode(
    theta=alt.Theta("Возраст:Q", stack=True),
    radius=alt.Radius("Возраст", scale=alt.Scale(type="sqrt", zero=True, rangeMin=20)),
    color="Дети:N",
)

c1 = base.mark_arc(innerRadius=20, stroke="#fff")

c2 = base.mark_text(radiusOffset=10).encode(text="Возраст:Q")

c1 + c2 

alt.LayerChart(...)


<div style='background: #C5E8EF; taxt-align: center; width: 444px; margin: auto; padding:44px; border: 1px solid #636363'>
    <h5 style='color: #476285;  text-align:center; margin: auto'>
        яркая картина того, что у людей семьей меньше времени на то, чтобы тратить его на социальные сети  
    </h5>
</div>